<a href="https://colab.research.google.com/github/chiangtinhung-rgb/Matcha/blob/main/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80HW2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
!pip install -q google-generativeai

In [66]:
!pip install gspread

In [67]:
!pip install gradio

In [68]:
import gradio as gr
import pandas as pd
from google.colab import auth
from google.auth import default

# -*- coding: utf-8 -*-
import gspread
from datetime import datetime
import google.generativeai as genai
import os
import json

In [69]:
from google.colab import userdata

# 從 Colab Secrets 中獲取 API 金鑰
api_key = userdata.get('MATCHA')

# 使用獲取的金鑰配置 genai
genai.configure(api_key=api_key)

model = genai.GenerativeModel('gemini-2.5-pro')



In [70]:
REQUIRED_COLUMNS = ["日期", "學期年", "科目", "成績", "預期成績", "備註"]
SHEET_URL = "https://docs.google.com/spreadsheets/d/10DefpZAIPlSIhR7b8DxSn_yqBw0U-YLap-cYggWlZGo/edit?gid=0#gid=0"
WORKSHEET_NAME = "工作表1"

_auth_done = False
_gc = None
_ws = None

In [71]:
try:
    auth.authenticate_user()
    creds, _ = default()
    gc = gspread.authorize(creds)
    sheets_enabled = True
    print("Google Sheets 認證成功。")
except Exception as e:
    print(f"Google Sheets 認證失敗: {e}")
    sheets_enabled = False
    gc = None

Google Sheets 認證成功。


In [72]:
# --- 主要功能區塊 ---
def get_user_grades():
    """
    透過終端機輸入學生成績，直到使用者輸入 'q' 結束。
    """
    print("--- 準備輸入成績。輸入 'q' 來停止。---")
    grades = []
    while True:
        subject = input("請輸入科目（或輸入 'q' 停止）：")
        if subject.lower() == 'q':
            break

        grade = input(f"請輸入 {subject} 的成績：")
        try:
            grade = int(grade)
        except ValueError:
            print("成績必須是數字。請重新輸入。")
            continue

        today = datetime.now().strftime('%Y-%m-%d')
        grades.append([today, subject, grade])
        print(f"已記錄：日期: {today}, 科目: {subject}, 成績: {grade}\n")

    return grades

In [73]:
def get_ai_subject_analysis(subject_record):
    """
    呼叫 Gemini 模型，對單一科目的成績記錄進行結構化分析，包含四個面向。

    Args:
        subject_record (dict): 包含單一成績記錄的字典。

    Returns:
        str: 包含結構化分析的 Markdown 文字。
    """
    if not MODEL_NAME or subject_record.get('成績') == 'N/A' or subject_record.get('預期成績') == 'N/A':
        return "⚠️ 資料不完整或 AI 服務配置錯誤，無法進行分析。"

    data_text = ""
    for k, v in subject_record.items():
        data_text += f"{k}: {v}\n"

    # 針對單科目的結構化分析提示，強制模型輸出四個結構化區塊
    system_prompt = (
        "您是一位專業且客觀的學業表現分析顧問。您的任務是根據單一科目的成績記錄，"
        "撰寫一份結構化、鼓勵性、且具有建設性的分析報告。請嚴格遵守以下輸出格式，"
        "並使用繁體中文。\n"
        "分析基於 '成績' 與 '預期成績' 之間的對比。"
    )

    user_query = (
        f"請根據以下成績記錄，生成一份詳細的學業表現分析報告：\n\n"
        f"--- 成績記錄 ---\n"
        f"{data_text}\n"
        f"-------------------\n\n"
        f"請嚴格按照以下 Markdown 格式輸出分析內容：\n\n"
        f"### 🎯 1. 達成標準與否 (成績 {subject_record.get('成績')} / 預期 {subject_record.get('預期成績')})\n"
        f"請判斷實際成績是否達到或超過預期成績。如果超過預期，請鼓勵；如果未達預期，請客觀指出差距。 (需具體提及數值)\n\n"
        f"### 🌟 2. 該成績評價與觀察\n"
        f"請客觀評價該科目這次的成績表現，並結合「備註」中的資訊（如果有的話）提出觀察到的學習狀況。評價應使用鼓勵和客觀的詞彙。\n\n"
        f"### 📈 3. 學習數據趨勢分析 (文字描述)\n"
        f"請以文字描述的方式，指出該筆成績（特別是成績與預期成績的差異）在長期趨勢圖表上可能代表的意義。建議應強調持續記錄的重要性。\n\n"
        f"### 💡 4. 回饋與具體改善建議\n"
        f"請提供 1-2 點針對性的、可操作的改善或保持優勢的建議。建議應針對性強，例如：「專注於計算題的步驟驗算」或「擴大英文閱讀量」。"
    )

    max_retries = 3
    for attempt in range(max_retries):
        try:
            response = genai.GenerativeModel(MODEL_NAME).generate_content(
                user_query,
                system_instruction=system_prompt,
            )
            return response.text
        except Exception as e:
            print(f"嘗試 {attempt + 1}/{max_retries} 呼叫 AI 失敗: {e}")
            if attempt == max_retries - 1:
                return f"AI 分析生成失敗：模型調用錯誤，請稍後再試。詳細錯誤：{e}"
    return "AI 分析生成失敗：無法連線到模型。"

In [74]:
def process_grades_and_analyze(grades_data):
    """
    處理 Gradio Dataframe 輸入的成績，並生成最終的 AI 綜合報告。
    """
    if not isinstance(grades_data, (pd.DataFrame, list)):
        return "錯誤：無效的成績數據格式。"

    # 轉換為 DataFrame
    if isinstance(grades_data, list):
        df = pd.DataFrame(grades_data, columns=REQUIRED_COLUMNS)
    else:
        df = grades_data

    # 篩選掉所有欄位都是空值的行
    df.dropna(how='all', inplace=True)

    if df.empty:
        return "請輸入至少一筆有效的成績記錄。"

    # 將 DataFrame 轉換為適合 AI 處理的記錄列表，並處理分數的類型轉換
    subject_records = df.to_dict('records')

    full_report = f"# 📚 學業表現綜合分析報告 ({datetime.now().strftime('%Y-%m-%d %H:%M:%S')})\n\n"
    full_report += f"本次分析共包含 **{len(subject_records)}** 筆成績記錄。\n"
    full_report += "--- \n"

    # 對每一筆成績記錄進行 AI 分析
    for i, record in enumerate(subject_records):
        subject = record.get('科目', '未知科目')

        # 類型轉換：確保成績欄位是數字，如果不是則標記為 N/A
        try:
            record['成績'] = int(record['成績'])
        except (ValueError, TypeError):
            record['成績'] = 'N/A'

        try:
            record['預期成績'] = int(record['預期成績'])
        except (ValueError, TypeError):
            record['預期成績'] = 'N/A'


        full_report += f"\n## 📝 科目分析: {subject} (第 {i+1} 筆記錄)\n"
        full_report += f"**日期:** {record.get('日期', 'N/A')} | **學期年:** {record.get('學期年', 'N/A')}\n\n"

        # 呼叫 AI 進行單科分析
        analysis_markdown = get_ai_subject_analysis(record)
        full_report += analysis_markdown
        full_report += "\n\n--- \n" # 分隔線

    return full_report


In [75]:
new_grades = get_user_grades()

--- 準備輸入成績。輸入 'q' 來停止。---
請輸入科目（或輸入 'q' 停止）：國文
請輸入 國文 的成績：90
已記錄：日期: 2025-10-05, 科目: 國文, 成績: 90

請輸入科目（或輸入 'q' 停止）：英文
請輸入 英文 的成績：95
已記錄：日期: 2025-10-05, 科目: 英文, 成績: 95

請輸入科目（或輸入 'q' 停止）：數學
請輸入 數學 的成績：70
已記錄：日期: 2025-10-05, 科目: 數學, 成績: 70

請輸入科目（或輸入 'q' 停止）：q


In [76]:
new_grades

[['2025-10-05', '國文', 90], ['2025-10-05', '英文', 95], ['2025-10-05', '數學', 70]]

In [79]:
# 主程式執行邏輯
def main():
    """
    主程式流程：輸入成績 -> 獲取 AI 摘要 -> 寫入 Google Sheet。
    """
    try:
        # 1. Google Sheet 身份驗證
        auth.authenticate_user()

        creds, _ = default()
        gc = gspread.authorize(creds)

        sh = gc.open_by_url(SHEET_URL)
        ws = sh.worksheet(WORKSHEET_NAME)





        print("--- Google Sheet 連線成功。---")

        # 2. 獲取使用者輸入的成績
        new_grades = get_user_grades()

        if not new_grades:
            print("沒有輸入任何成績，程式結束。")
            return

        # 3. 將新成績寫入 Google Sheet
        ws.append_rows(new_grades)
        print("\n--- 成績已成功寫入 Google Sheet。---")

        # 4. 獲取 AI 摘要並寫入 Google Sheet
        summary = get_ai_summary(new_grades)

        # 尋找第一行空白列
        next_row = len(ws.col_values(1)) + 1

        # 使用 update_cell() 方法逐一更新儲存格
        ws.update_cell(next_row, 1, datetime.now().strftime('%Y-%m-%d'))
        ws.update_cell(next_row, 2, 'AI 摘要')

        # 為了避免單元格內容過長，將摘要內容分成多行來寫入
        summary_lines = summary.split('\n')
        for i, line in enumerate(summary_lines):
            ws.update_cell(next_row + i, 3, line)

        print("\n--- AI 摘要已成功寫入 Google Sheet。---")
        print("以下是 AI 生成的摘要內容：")
        print("-" * 50)
        print(summary)
        print("-" * 50)

    except gspread.exceptions.APIError as e:
        print(f"Google Sheets API 錯誤：{e.response.text}")
        print("請確認：")
        print("1. 您的服務帳戶金鑰檔案正確且未過期。")
        print("2. 您已將服務帳戶的 Email 地址（在 JSON 檔案中）分享給 Google Sheet，並給予編輯權限。")
    except Exception as e:
        print(f"發生未預期的錯誤：{e}")

if __name__ == "__main__":
    main()


--- Google Sheet 連線成功。---
--- 準備輸入成績。輸入 'q' 來停止。---
請輸入科目（或輸入 'q' 停止）：國文
請輸入 國文 的成績：90
已記錄：日期: 2025-10-05, 科目: 國文, 成績: 90

請輸入科目（或輸入 'q' 停止）：英文
請輸入 英文 的成績：95
已記錄：日期: 2025-10-05, 科目: 英文, 成績: 95

請輸入科目（或輸入 'q' 停止）：數學
請輸入 數學 的成績：70
已記錄：日期: 2025-10-05, 科目: 數學, 成績: 70

請輸入科目（或輸入 'q' 停止）：q

--- 成績已成功寫入 Google Sheet。---
發生未預期的錯誤：name 'get_ai_summary' is not defined


In [80]:
# --- 圖表函數 (已更名為 plot_subject_trend 以匹配 Gradio 配置) ---

def plot_subject_trend(df: pd.DataFrame, selected_subject: str, start_date: str, end_date: str):
    """
    繪製單一科目的成績趨勢圖。
    """
    # 檢查數據是否已載入
    if df is None or df.empty or '日期' not in df.columns or '成績' not in df.columns:
        fig, ax = plt.subplots()
        ax.text(0.5, 0.5, '請先載入 Google Sheets 數據', ha='center', va='center', fontsize=12)
        ax.axis('off')
        return fig

    # 1. 過濾科目
    subject_df = df[df['科目'] == selected_subject].copy()

    # 2. 過濾日期
    if start_date and start_date.strip():
        try:
            subject_df = subject_df[subject_df['日期'] >= pd.to_datetime(start_date)]
        except ValueError:
             print(f"警告：起始日期格式錯誤: {start_date}")

    if end_date and end_date.strip():
        try:
            # 確保結束日期包含整天
            subject_df = subject_df[subject_df['日期'] <= pd.to_datetime(end_date) + pd.Timedelta(days=1)]
        except ValueError:
             print(f"警告：結束日期格式錯誤: {end_date}")

    if subject_df.empty:
        fig, ax = plt.subplots()
        ax.text(0.5, 0.5, f'在選定的日期範圍內，科目 "{selected_subject}" 沒有數據。', ha='center', va='center', fontsize=12)
        ax.axis('off')
        return fig

    # 3. 繪製圖表
    fig, ax = plt.subplots(figsize=(10, 6))

    # 計算平均值
    mean_grade = subject_df['成績'].mean()

    # 主圖：成績點和線
    ax.plot(subject_df['日期'], subject_df['成績'], marker='o', linestyle='-', color='#007ACC', linewidth=2, label='每次成績')
    ax.scatter(subject_df['日期'], subject_df['成績'], s=100, color='#007ACC', zorder=5)

    # 輔助線：平均值
    ax.axhline(mean_grade, color='#FF5733', linestyle='--', linewidth=1.5, label=f'平均分 ({mean_grade:.1f})')

    # 設定標籤和標題
    ax.set_title(f'{selected_subject} 成績趨勢分析', fontsize=16, fontweight='bold')
    ax.set_xlabel('日期', fontsize=12)
    ax.set_ylabel('成績 (分數)', fontsize=12)

    # 格式化 X 軸日期
    fig.autofmt_xdate(rotation=45)

    # Y 軸範圍固定在 0-100
    ax.set_ylim(0, 100)

    # 增加網格線
    ax.grid(True, linestyle='--', alpha=0.7)

    # 顯示圖例
    ax.legend(loc='best')

    plt.close(fig) # 關閉圖表以防內存洩漏
    return fig




In [81]:
# ----------------------------------------------------------------------
# 圖表分析
# ----------------------------------------------------------------------

def plot_subject_performance(df, selected_subject, start_date=None, end_date=None):
    """
    繪製單一科目的成績趨勢圖。
    """
    if df is None or df.empty:
        # 返回一個空的 Matplotlib 圖
        fig, ax = plt.subplots()
        ax.text(0.5, 0.5, '請先解析成績數據', ha='center', va='center')
        ax.axis('off')
        return fig

    # 1. 過濾科目
    subject_df = df[df['科目'] == selected_subject].copy()

    # 2. 過濾日期 (實現「查詢時間自由選擇」)
    if start_date and start_date.strip(): # 檢查是否為空字串或 None
        try:
            subject_df = subject_df[subject_df['日期'] >= pd.to_datetime(start_date)]
        except ValueError:
             print(f"警告：起始日期格式錯誤: {start_date}")

    if end_date and end_date.strip(): # 檢查是否為空字串或 None
        try:
            # 確保結束日期包含整天
            subject_df = subject_df[subject_df['日期'] <= pd.to_datetime(end_date) + pd.Timedelta(days=1)]
        except ValueError:
             print(f"警告：結束日期格式錯誤: {end_date}")


    if subject_df.empty:
        fig, ax = plt.subplots()
        ax.text(0.5, 0.5, f'在選定的日期範圍內，科目 "{selected_subject}" 沒有數據。', ha='center', va='center', fontsize=12)
        ax.axis('off')
        return fig

    # 3. 繪製圖表
    fig, ax = plt.subplots(figsize=(10, 6))

    # 計算平均值和中位數
    mean_grade = subject_df['成績'].mean()
    median_grade = subject_df['成績'].median()

    # 主圖：成績點和線
    ax.plot(subject_df['日期'], subject_df['成績'], marker='o', linestyle='-', color='#007ACC', linewidth=2, label='每次成績')
    ax.scatter(subject_df['日期'], subject_df['成績'], s=100, color='#007ACC', zorder=5)

    # 輔助線：平均值和中位數
    ax.axhline(mean_grade, color='#FF5733', linestyle='--', linewidth=1.5, label=f'平均分 ({mean_grade:.1f})')
    ax.axhline(median_grade, color='#FFA500', linestyle=':', linewidth=1.5, label=f'中位數 ({median_grade:.1f})')

    # 設定標籤和標題
    ax.set_title(f'{selected_subject} 成績趨勢分析', fontsize=16, fontweight='bold')
    ax.set_xlabel('日期', fontsize=12)
    ax.set_ylabel('成績 (分數)', fontsize=12)

    # 格式化 X 軸日期
    fig.autofmt_xdate(rotation=45)

    # Y 軸範圍固定在 0-100
    ax.set_ylim(0, 100)

    # 增加網格線
    ax.grid(True, linestyle='--', alpha=0.7)

    # 顯示圖例
    ax.legend(loc='best')

    plt.close(fig) # 關閉圖表以防內存洩漏
    return fig




In [82]:
# --- 助手函數：從 DataFrame 中獲取唯一的科目列表 ---
def get_subjects_from_df(df):
    if df.empty or '科目' not in df.columns:
        return ["--- 載入數據後選擇 ---"]

    subjects = df['科目'].dropna().unique().tolist()
    return subjects if subjects else ["--- 載入數據後選擇 ---"]

In [ ]:
# --- Gradio 介面配置 ---
with gr.Blocks(theme=gr.themes.Soft(), title="學業成績分析儀") as demo:
    gr.Markdown("# 🎓 學業成績追蹤與 AI 分析儀")
    gr.Markdown("---")

    # 數據緩存組件 (不可見)
    df_data = gr.State(pd.DataFrame(columns=REQUIRED_COLUMNS))
    connection_status = gr.State("未連線") # 新增 State 儲存連接狀態，用於條件檢查

    with gr.Tab("💾 數據操作與管理"):
        gr.Markdown("## 步驟 1: Google Sheets 數據載入")
        connect_btn = gr.Button("🔗 嘗試連接 Google Sheets (Colab 環境限定)", variant="secondary")
        connect_status = gr.Textbox(label="連線狀態", value="未連線，請點擊按鈕或在 Colab 環境中運行", interactive=False)
        load_btn = gr.Button("⬇️ 載入所有成績數據 (用於圖表分析)", variant="primary")
        sheet_data_display = gr.Dataframe(
            label="已載入的 Google Sheet 數據 (僅供查看)",
            headers=REQUIRED_COLUMNS,
            datatype=["str", "str", "str", "number", "number", "str"],
            row_count=5,
            col_count=len(REQUIRED_COLUMNS),
            interactive=False
        )
        load_status = gr.Textbox(label="載入狀態", interactive=False)

        # 連接按鈕點擊時，只更新狀態文字
        connect_btn.click(connect_to_sheets, outputs=[connect_status])

        # 點擊載入數據時，更新 Dataframe State 和 Dataframe 顯示
        load_btn.click(
            load_data_from_sheet,
            outputs=[df_data, load_status]
        )

        # 使用一個小的副作用函數來更新可視的 Dataframe 顯示
        def update_display_df(df):
            # 確保日期欄位在顯示時是可讀的字串
            if '日期' in df.columns and pd.api.types.is_datetime64_any_dtype(df['日期']):
                 df['日期'] = df['日期'].dt.strftime('%Y-%m-%d')
            return df

        df_data.change(update_display_df, inputs=df_data, outputs=sheet_data_display)


    with gr.Tab("📝 新成績輸入與 AI 分析"):
        gr.Markdown("## 步驟 2: 輸入新成績並進行 AI 結構化分析")
        gr.Markdown(
            "請在下方表格輸入新的成績記錄，然後點擊 **[執行 AI 分析]** 按鈕獲取詳細報告。  "
            "輸入完成並確認無誤後，可點擊 **[寫入 Google Sheet]** 保存數據。"
        )

        # 預設值為 6 列
        input_df = gr.Dataframe(
            label="輸入成績表格 (可編輯)",
            headers=REQUIRED_COLUMNS,
            datatype=["str", "str", "str", "number", "number", "str"],
            value=[
                [datetime.now().strftime('%Y-%m-%d'), "2024上", "國文", 85, 90, "期中考，文言文部分失分較多"],
                [datetime.now().strftime('%Y-%m-%d'), "2024上", "數學", 92, 85, "計算錯誤導致扣分，概念理解無誤"],
            ],
            row_count=5,
            col_count=len(REQUIRED_COLUMNS),
            interactive=True
        )

        with gr.Row():
            analyze_btn = gr.Button("🧠 執行 AI 分析", variant="primary")
            write_btn = gr.Button("⬆️ 寫入 Google Sheet", variant="secondary")
            write_status = gr.Textbox(label="寫入狀態", value="未寫入", scale=2, interactive=False)

        ai_report = gr.Markdown(label="AI 學業表現報告", value="點擊 [執行 AI 分析] 以生成報告。")

        analyze_btn.click(
            process_grades_and_analyze,
            inputs=input_df,
            outputs=ai_report
        )

        # 寫入功能: 只寫入 Dataframe 的內容
        write_btn.click(
            append_data_to_sheet,
            inputs=input_df,
            outputs=[write_status]
        )

    with gr.Tab("📊 趨勢圖表分析"):
        gr.Markdown("## 步驟 3: 成績趨勢圖表")

        # 助手函數用於動態更新科目下拉菜單
        def update_subject_dropdown(df):
            subjects = get_subjects_from_df(df)
            return gr.Dropdown(choices=subjects, value=subjects[0] if subjects else None, interactive=True)

        with gr.Row():
            subject_dropdown = gr.Dropdown(
                label="選擇科目",
                choices=["--- 載入數據後選擇 ---"],
                value="--- 載入數據後選擇 ---",
                interactive=False,
                scale=1
            )
            start_date_input = gr.Textbox(label="起始日期 (YYYY-MM-DD)", placeholder="留空則不限制", scale=1)
            end_date_input = gr.Textbox(label="結束日期 (YYYY-MM-DD)", placeholder="留空則不限制", scale=1)
            plot_btn = gr.Button("🔄 繪製/更新圖表", variant="primary", scale=0)

        trend_plot = gr.Plot(label="成績趨勢圖")

        # 繪製圖表邏輯
        plot_btn.click(
            plot_subject_trend,
            inputs=[df_data, subject_dropdown, start_date_input, end_date_input],
            outputs=trend_plot
        )

        # 當 df_data (State) 發生變化時，動態更新科目下拉選單
        df_data.change(update_subject_dropdown, inputs=df_data, outputs=subject_dropdown)

    gr.Markdown("---")
    gr.Markdown("💡 **備註:** AI 分析需要有效的 Gemini API Key (透過 Colab Secret `MATCHA`)。數據操作功能需要在 Google Colab 環境中執行 `auth.authenticate_user()`。")

# 運行 Gradio 介面
if __name__ == "__main__":

    # 設置 Matplotlib 默認字體，以確保中文顯示正常
    try:
        # 假設環境中有支援中文的字體
        plt.rcParams['font.sans-serif'] = ['Microsoft YaHei', 'SimHei', 'Taipei Sans TC Beta']
        plt.rcParams['axes.unicode_minus'] = False # 解決負號亂碼問題
    except Exception:
        pass # 如果環境中沒有這些字體，則忽略

    demo.launch(debug=True)

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://b4493b3a5f2c9cb8f6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Attempting to connect to Google Sheets...
Loading data from sheet...
Loading data from sheet...
Attempting to connect to Google Sheets...
Attempting to connect to Google Sheets...
